In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Overview
This notebook uses a CNN with 4 convolutional layers to distinguish between photos of dogs and cats.

Training accuracy after 25 epochs is about 96% while validation accuracy is around 89% so there is some overfitting going on.

The images come in different sizes and originally an ImageDataGenerator was used to resize to a fixed size 150 x 150.  The ImageDataGenerator was also used for rescaling of the pixel value by dividing by 255.  It was noticed that the CPU bottlenecked the training process, which took some 75 seconds per epoch with GPU acceleration.

Since the repetitive resizing-on-the-fly is performed by the CPU and therefore contributed to the CPU bottleneck, code was added to separately resize the source images before training and store the results to disk.  The training algorithm was then applied to the resized images.  This is the current flow, with the ImageDataGenerator still performing the pixel rescaling.  The CPU still bottlenecks training but training time has been reduced to 26 seconds per epoch, with as well as without GPU accelaration.

The remaining CPU bottlenecking is likely caused by the repetitive fetching of data and training of tha data, a flow that can be further optimized.

In [ ]:
%matplotlib inline

import zipfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil

import tensorflow as tf
from tensorflow.keras import utils
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

Check for availability and use of GPU.

In [ ]:
!nvidia-smi
print(tf.test.is_gpu_available())
print(tf.test.gpu_device_name())
print(tf.config.experimental.list_physical_devices(device_type='GPU'))

Empty the /kaggle/working directory so you can run this notebook multiple times.

In [ ]:
working_dir = "/kaggle/working/"
for rootdir, dirs, files in os.walk(working_dir):
    for subdir in dirs:
        shutil.rmtree(subdir)

Unzip the training set in the working directory.

In [ ]:
input_dir = "/kaggle/input/dogs-vs-cats/"

local_zip = input_dir + 'train.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(working_dir)
zip_ref.close()

Unzip the test set.

In [ ]:
local_zip = input_dir + 'test1.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(working_dir)
zip_ref.close()

Do not try to see the files in the training or test set in the data panel on the right as it will hang the browser.

Print the names of a few files in the training set.

In [ ]:
train_dir = working_dir + 'train/'
for dirname, _, filenames in os.walk(train_dir):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

Print the names of a few files in the test set.

In [ ]:
test_dir = working_dir + 'test1/'
for dirname, _, filenames in os.walk(test_dir):
    for filename in filenames[0:3]:
        print(os.path.join(dirname, filename))

Plot a few training images.

In [ ]:
image_files = ["/kaggle/working/train/dog.918.jpg",
               "/kaggle/working/train/dog.7156.jpg",
               "/kaggle/working/train/dog.10335.jpg"]

for i, img_dir in enumerate(image_files):
  img = mpimg.imread(img_dir)
  plt.imshow(img)
  plt.axis('Off')
  plt.show()

Print the number of training and test images.

In [ ]:
print(len(os.listdir(train_dir)))
print(len(os.listdir(test_dir)))

# Separate Resizing
Resize the training images and put the results in a new training directory.

In [ ]:
destination_dir = working_dir + 'train_resized/'
if (os.path.isdir(destination_dir) == False):
    os.mkdir(destination_dir)

for dirname, _, filenames in os.walk(train_dir):
    for filename in filenames:
        input_path = os.path.join(dirname, filename)
        img = image.load_img(input_path, target_size=(150, 150))
        x = image.img_to_array(img)
#         x = np.divide(x, 255.0)
        output_path = os.path.join(destination_dir, filename)
        utils.save_img(output_path, x, data_format='channels_last', file_format='jpeg', scale=False)
        
train_dir = working_dir + 'train_resized/'

Move the training images to separate subfolders so we can use an ImageDataGenerator and have it automatically create the labels from the subfolder names.

In [ ]:
dog_files = [name for name in os.listdir(train_dir) if (name.split('.')[0]=='dog')]
print(dog_files[:3])
print(len(dog_files))
cat_files = [name for name in os.listdir(train_dir) if (name.split('.')[0]=='cat')]
print(cat_files[:3])
print(len(cat_files))

In [ ]:
train_subs_dir = working_dir + 'train_subs/'
if (os.path.isdir(train_subs_dir) == False):
    os.mkdir(train_subs_dir)

train_subs_dog_dir = train_subs_dir + 'dog/'
if (os.path.isdir(train_subs_dog_dir) == False):
    os.mkdir(train_subs_dog_dir)

train_subs_cat_dir = train_subs_dir + 'cat/'
if (os.path.isdir(train_subs_cat_dir) == False):
    os.mkdir(train_subs_cat_dir)

print(train_subs_dir)
print(train_subs_dog_dir)
print(train_subs_cat_dir)

for name in dog_files:
    shutil.copyfile(train_dir + name, train_subs_dog_dir + name)
    
for name in cat_files:
    shutil.copyfile(train_dir + name, train_subs_cat_dir + name)

In [ ]:
for dirname, _, filenames in os.walk(train_subs_dog_dir):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

for dirname, _, filenames in os.walk(train_subs_cat_dir):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

In [ ]:
print(len(os.listdir(train_subs_dog_dir)))
print(len(os.listdir(train_subs_cat_dir)))

Use the last 2500 dog images and the last 2500 cat images as the validation set.

In [ ]:
val_subs_dir = working_dir + 'val_subs/'
if (os.path.isdir(val_subs_dir) == False):
    os.mkdir(val_subs_dir)

val_subs_dog_dir = val_subs_dir + 'dog/'
if (os.path.isdir(val_subs_dog_dir) == False):
    os.mkdir(val_subs_dog_dir)

val_subs_cat_dir = val_subs_dir + 'cat/'
if (os.path.isdir(val_subs_cat_dir) == False):
    os.mkdir(val_subs_cat_dir)

In [ ]:
for name in dog_files[:2500]:
    os.rename(train_subs_dog_dir + name, val_subs_dog_dir + name)
for name in cat_files[:2500]:
    os.rename(train_subs_cat_dir + name, val_subs_cat_dir + name)

In [ ]:
print(train_subs_dog_dir)
print(len(os.listdir(train_subs_dog_dir)))
for dirname, _, filenames in os.walk(train_subs_dog_dir):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

print(train_subs_cat_dir)
print(len(os.listdir(train_subs_cat_dir)))
for dirname, _, filenames in os.walk(train_subs_cat_dir):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

print(val_subs_dog_dir)
print(len(os.listdir(val_subs_dog_dir)))
for dirname, _, filenames in os.walk(val_subs_dog_dir):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

print(val_subs_cat_dir)
print(len(os.listdir(val_subs_cat_dir)))
for dirname, _, filenames in os.walk(val_subs_cat_dir):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

Create the image data generators.

In [ ]:
train_subs_dir = working_dir + 'train_subs/'
val_subs_dir = working_dir + 'val_subs/'

training_datagen = ImageDataGenerator(
      rescale = 1./255,
#       rotation_range=20,
#       width_shift_range=0.2,
#       height_shift_range=0.2,
#       shear_range=0.2,
#       zoom_range=0.2,
#       horizontal_flip=True,
#       fill_mode='nearest'
)

train_generator = training_datagen.flow_from_directory(
    train_subs_dir,
    target_size=(150,150),
    class_mode='binary',
    batch_size=128
)

validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_generator = validation_datagen.flow_from_directory(
    val_subs_dir,
    target_size=(150,150),
    class_mode='binary',
    batch_size=128
)

Create the model.

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.summary()

Compile the model.

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# model.compile(optimizer = RMSprop(learning_rate=0.0001), 
#                 loss = 'binary_crossentropy',
#                 metrics = ['accuracy'])

Create callbacks.

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.99):
      print("\nReached 99% accuracy, cancelling training")
      self.model.stop_training = True

Train the model.

In [ ]:
callbacks = myCallback()
model.fit(train_generator,
          validation_data = validation_generator,
          epochs=25,
          batch_size=128,
          verbose = 2,
          validation_steps=3,
          callbacks=callbacks
         )

model.save("rps.h5")

# Results
## No separate resizing, no acceleration
Training duration per epoch: unknown, at least 6 minutes.

## No separate resizing, GPU acceleration
Training duration per epoch: ~75 seconds

## Separate resizing, no acceleration
Training duration per epoch: ~31 seconds

## Separate resizing, GPU acceleration
Training duration per epoch: ~31 seconds

In [ ]:
print(len(os.listdir(test_dir)))
test_files = os.listdir(test_dir)
print(test_files[:3])
id_strings = [name.split('.')[0] for name in os.listdir(test_dir)]
test_ids = list(map(int, id_strings))
test_ids.sort()
# print(test_ids) # it looks like test_ids - ie. the numbers in the test image file names - is simply a list from 1 thru 12500
print(min(test_ids))
print(max(test_ids))
sorted_id_strings = [(test_dir+str(test_id)+'.jpg') for test_id in test_ids]
print(sorted_id_strings[:10])

In [ ]:
for name in sorted_id_strings[:10]:
    img = image.load_img(name, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.divide(x, 255.0)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict(images)
    print(name)
    print(classes[0][0])
    if classes[0][0]>0.5:
      print("Dog")
    else:
      print("Cat")
    plt.imshow(img)
    plt.axis('Off')
    plt.show()
    print()

In [ ]:
predictions = []

for name in sorted_id_strings:
    img = image.load_img(name, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.divide(x, 255.0)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict(images)
    if classes[0][0]>0.5:
      prediction = 1 # Dog
    else:
      prediction = 0 # Cat
    predictions.append(prediction)

In [ ]:
print(len(predictions))
df = pd.DataFrame()
df['id'] = test_ids
df['label'] = predictions
print(df)

In [ ]:
df.to_csv('submission.csv', index=False)